In [ ]:
from llama_index.llms.groq import Groq
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
import numpy as np

In [ ]:
from llama_index.core import download_loader, Document, VectorStoreIndex, ServiceContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine.transform_query_engine import TransformQueryEngine
from llama_index.core.response.notebook_utils import display_source_node

In [ ]:
from pathlib import Path


In [ ]:
# Load PDF data
PDFReader = download_loader("PDFReader")
loader = PDFReader()
docs = loader.load_data(file=Path("/content/nasa-hubble-focus-dark-universe-2024-apr-v2.pdf"))

# Create chunks
node_parser = SentenceSplitter(chunk_size=256)
nodes = node_parser.get_nodes_from_documents(docs)

<ipython-input-11-93c3e9e7746f>:2: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PDFReader = download_loader("PDFReader")


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embeddings = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Initialize LLM
llm = Groq(model="llama3-70b-8192", api_key="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")


In [ ]:
# Create ServiceContext
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings)


<ipython-input-15-19e7d5207d02>:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings)


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(
    url="https://a39527c9-207f-4db4-82b9-6d8039f3c72d.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="KaqAcYuLeNIvUxWMQJisJrqcm2rERYB381zqbuWWz_0pFxB2rkZ18A",
)

In [ ]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
vector_store = QdrantVectorStore(qdrant_client=client, collection_name="text_db",
                                  url="https://a39527c9-207f-4db4-82b9-6d8039f3c72d.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="KaqAcYuLeNIvUxWMQJisJrqcm2rERYB381zqbuWWz_0pFxB2rkZ18A", )


In [ ]:
# Create index with Qdrant vector store
vector_index = VectorStoreIndex(nodes, service_context=service_context, vector_store=vector_store)


**********
Trace: index_construction
    |_embedding -> 1.502797 seconds
    |_embedding -> 1.45104 seconds
    |_embedding -> 1.41235 seconds
    |_embedding -> 1.537631 seconds
    |_embedding -> 2.728457 seconds
    |_embedding -> 2.929795 seconds
    |_embedding -> 1.325105 seconds
    |_embedding -> 2.449627 seconds
    |_embedding -> 2.8229 seconds
    |_embedding -> 3.842397 seconds
    |_embedding -> 3.472553 seconds
    |_embedding -> 2.664392 seconds
    |_embedding -> 2.739533 seconds
    |_embedding -> 2.534423 seconds
    |_embedding -> 3.19496 seconds
    |_embedding -> 3.489062 seconds
    |_embedding -> 2.687873 seconds
    |_embedding -> 2.407007 seconds
    |_embedding -> 3.708795 seconds
    |_embedding -> 4.806443 seconds
    |_embedding -> 2.329477 seconds
    |_embedding -> 3.057674 seconds
    |_embedding -> 2.429339 seconds
    |_embedding -> 3.023195 seconds
    |_embedding -> 3.874959 seconds
    |_embedding -> 3.12233 seconds
    |_embedding -> 2.567871 secon

In [ ]:
# Query the engine
query_str = "How has Hubble contributed to our understanding of dark matter, and what implications do these contributions have for future astronomical research?"


In [ ]:
query_engine = vector_index.as_query_engine()
response = query_engine.query(query_str)


In [ ]:
from IPython.display import Markdown, display
display(Markdown(f"{response}"))

Hubble has contributed to our understanding of dark matter by discovering small clumps of it, which supports one of the prevailing theories about its nature. However, Hubble's observations have also revealed inconsistencies with theoretical models, suggesting that there are still missing pieces to the puzzle. Despite these inconsistencies, Hubble's continued observations, in tandem with other telescopes, will help trace the distribution of dark matter, ultimately shedding light on its composition. Uncovering the true nature of dark matter will have significant implications for predicting its effect on the evolution of galaxies, galaxy clusters, and the universe as a whole.

In [ ]:
# Apply nested asyncio
import nest_asyncio
nest_asyncio.apply()

# Using the LlamaDebugHandler to print the trace of the sub questions captured by the SUB_QUESTION callback event type
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

# Create ServiceContext with callback manager
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings, callback_manager=callback_manager)


# Initialize vector query engine
vector_query_engine = vector_index.as_query_engine(use_async=True)

<ipython-input-42-6009cc1c6c80>:12: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings, callback_manager=callback_manager)


In [ ]:


# Define query engine tools
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="dark_energy_expansion",
            description="Insights and analysis on dark energy and the expansion of the universe",
        ),
    ),
]

# Create SubQuestionQueryEngine
query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)


In [ ]:

# Query the engine
query_str = "How has Hubble contributed to our understanding of dark matter and what implications do these contributions have for future astronomical research?"

response = query_engine.query(query_str)



In [ ]:

# Query the engine
query_str = "What is dark energy, and why is it described as weak on smaller scales (such as within galaxies or solar systems) but noticeable on intergalactic scales, particularly in contrast to the effects of gravity on cosmic scales?"

response = query_engine.query(query_str)



In [ ]:
display(Markdown(f"{response}"))

The Hubble Space Telescope data has revealed inconsistencies in the current cosmological model by showing an observed expansion rate of the universe that is higher than the predicted rate, with a significant discrepancy between the two values, and the small uncertainties in these numbers do not overlap, affirming the discrepancy.